# Recommender System

# Things to fix in eda:
    1) wordsstucktogether
    2) _________
    3) ??????????
    4) **********
    5) random numbers

In [147]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import spacy

---

In [148]:
jobs = pd.read_csv('../data/job_postings.csv')
jobs = jobs.drop(columns=['date_added', 'organization', 'skills_len', 'job_type'])

In [149]:
jobs.head(5)

,job_description,job_title,location,skills
0,"If you are an EDI Analyst with experience, ple...",Analyst,Northeast United States,edi trustedlink as2 van
1,"Informatica/ ETL DeveloperSt, Petersburg, FL O...",Developer,Southern United States,etl informatica b2b data exchange netezza orac...
2,"Sunnyvale , CAANGULARJSdeveloperswithhibernat...",Developer,Western United States,angular
3,This nationally recognized Microsoft Gold Part...,Manager,Western United States,microsoft dynamics ax project manager - toront...
4,"If you are a .NET Developer with experience, p...",Developer,Northeast United States,c# asp.net sql javascript mvc


In [150]:
jobs.isna().sum()

job_description      6
job_title            0
location             0
skills             390
dtype: int64

In [151]:
# Replace nan's with empty string
jobs.fillna('', inplace=True)

In [152]:
# Combine job description and text into a single column
jobs['text'] = jobs['job_description'] + ' ' + jobs['skills']

# Tf-Idf
https://www.datacamp.com/community/tutorials/recommender-systems-python

In [34]:
# Vectorize the training data
tfidf = TfidfVectorizer(stop_words='english')

tfidf_matrix = tfidf.fit_transform(jobs['text'])

tfidf_matrix.shape

(16661, 94943)

In [44]:
# Reminder of what this looks like
tfidf_matrix.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [42]:
# Looking at the 'words' that show up in the training data
tfidf.get_feature_names()[15000:15010]

['businesstrain',
 'businesstranslates',
 'businesstroubleshoot',
 'businesstypically',
 'businessunderstand',
 'businessweb',
 'businessweek',
 'businesswhat',
 'businesswillingness',
 'businessworked']

In [50]:
# Vectorize the user's profile
tfidf_user = TfidfVectorizer(stop_words='english')

user_input_tfidf = tfidf_user.fit_transform(['java developer'])

user_input_tfidf.shape

(1, 2)

In [53]:
# Calculate the cosine similarity
# cosine_sim = linear_kernel(tfidf_matrix, user_input_tfidf)

# Spacy
https://spacy.io/usage/spacy-101

In [153]:
# Load one of the larger models for a better similarity score
nlp = spacy.load("en_core_web_lg")

In [154]:
# How to use .similarity function
tokens = nlp("dog cat banana")

for token1 in tokens:
    for token2 in tokens:
        print(token1.text, token2.text, token1.similarity(token2))

dog dog 1.0
dog cat 0.8016855
dog banana 0.24327648
cat dog 0.8016855
cat cat 1.0
cat banana 0.28154367
banana dog 0.24327648
banana cat 0.28154367
banana banana 1.0


In [155]:
# Some more examples
print(nlp("dog").similarity(nlp("dog")))
print(nlp("dog").similarity(nlp("cat")))
print(nlp("dog").similarity(nlp("banana")))

1.0
0.8016855517329495
0.24327647954195658


In [156]:
# Displaying some attributes of an nlp object
tokens = nlp("dog cat banana afskfsd")

for token in tokens:
    print(token.text, token.has_vector, token.vector_norm, token.is_oov)

dog True 7.0336733 False
cat True 6.6808186 False
banana True 6.700014 False
afskfsd False 0.0 True


---
## Using our jobs data

In [157]:
titles_and_docs = jobs[['job_title']].copy()
titles_and_docs['doc'] = jobs['text'].map(nlp)

In [158]:
titles_and_docs

,job_title,doc
0,Analyst,"(If, you, are, an, EDI, Analyst, with, experie..."
1,Developer,"(Informatica/, ETL, DeveloperSt, ,, Petersburg..."
2,Developer,"( , Sunnyvale, ,, CAANGULARJSdeveloperswithhib..."
3,Manager,"(This, nationally, recognized, Microsoft, Gold..."
4,Developer,"(If, you, are, a, .NET, Developer, with, exper..."
...,...,...
16656,Developer,"(Job, Description, , JPMorgan, Chase, &, Co.,..."
16657,Administrator,"(Seeking, Jr., Systems, Administrators, with, ..."
16658,Developer,"(If, you, are, a, Senior, /, Lead, Devops, Eng..."
16659,Developer,"(Headquartered, in, downtown, San, Francisco, ..."


In [175]:
# Comparing job postings to the single word 'Analyst'
# Highest similarity score in the first 2000 posts is .514
doc = nlp('Analyst')

max(titles_and_docs['doc'][:2000].map(lambda x: x.similarity(doc)))

0.5141672540267872

In [187]:
# Load in profile data to test the similarity score
profile_data_zach = pd.read_csv('../data/linkedin/test-output/Zach_LinkedInData_12-16-2020.csv')

profile_data_zach['text'] = profile_data_zach['Titles'] + ' ' \
                        + profile_data_zach['Skills'] + ' ' \
                        + profile_data_zach['Summary'] + ' ' \
                        + profile_data_zach['Education']

profile_data_zach['text'][0]

"Data Science Fellow, Python Developer, Health and Fitness Coach, IT Specialist, Data Analyst, Computational Modeling, Data Analyst Data Analysis, Python (Programming Language), Git, Data Visualization, Statistics, Machine Learning, Microsoft Excel, Microsoft Office, SQL, GitHub, Microsoft Azure, Team Leadership I bridge the gap between data and climate policy as a data scientist.  I work with climate scientists and the data they collect to extract insights and ensure appropriate action is taken by policy makers. My favorite activity is an ocean plunge, any time, any weather. This motivates me to inform and guide decision making towards a healthy relationship with our natural world.  Skills: Python, machine learning, SQL, data cleaning and preprocessing, ETL, Microsoft Azure, dashboard reporting, web application development. Data Science Intensive, Bachelor's of Science"

In [188]:
# Zach's profile
profile_text_zach = profile_data_zach['text'][0]
profile_doc_zach = nlp(profile_text_zach)

# First similarity scores for first 100 job postings
titles_and_docs['doc'][:100].map(lambda x: x.similarity(profile_doc_zach))

0     0.909253
1     0.910925
2     0.704195
3     0.925627
4     0.925101
        ...   
95    0.905330
96    0.942924
97    0.940148
98    0.919232
99    0.910125
Name: doc, Length: 100, dtype: float64

In [197]:
# Calculate all similarity scores
zach_scores = jobs[['job_title']].copy()
zach_scores['sim_score'] = titles_and_docs['doc'].map(lambda x: x.similarity(profile_doc_zach))

In [198]:
# Zach's results
zach_scores.groupby('job_title').agg(['count', 'mean'])['sim_score'].sort_values('mean', ascending=False)

,count,mean
job_title,,
Data Position,283,0.926926
Analyst,1900,0.925358
Engineer,4118,0.924823
Architect,831,0.923081
Programmer,321,0.920916
Director,146,0.920628
Administrator,899,0.920092
Developer,5274,0.919908
Manager,1390,0.918922


In [193]:
# Load in profile data to test the similarity score
profile_data_nolan = pd.read_csv('../data/linkedin/test-output/Nolan_LinkedInData_12-16-2020.csv')

profile_data_nolan['text'] = profile_data_nolan['Titles'] + ' ' \
                        + profile_data_nolan['Skills'] + ' ' \
                        + profile_data_nolan['Summary'] + ' ' \
                        + profile_data_nolan['Education']

profile_data_nolan['text'][0]

"Data Science Fellow, Painter Data Science, Python, Data Analysis, Data Management, Business Intelligence (BI), Analytical Skills, Web Development, Business Analytics, Wireframing, Pivot Tables, Identifying New Opportunities, Data Visualization, Machine Learning, SQL, Microsoft Excel, HTML, Tableau, JavaScript, Microsoft PowerPoint, Microsoft Word, Windows, SQL Server Management Studio, Tenable Nessus, Microsoft Office, Microsoft SQL Server, GitHub, Git, Organizational Leadership, Interpersonal Communication, Collaborative Problem Solving, Data Cleaning, Online Social Networking, Wireshark, NetWitness, Predictive Modeling, Command Line, Web Scraping An innovative Data Scientist who is passionate about constructing programs that are more efficient utilizing functions for automation. With a background in management information systems I bring an analytical, calculated process to any organization.  Technologies: SQL | HTML5 | Javascript | AWS EC2 | Jupyter Notebooks | Github | Git | Excel

In [194]:
# Nolan's profile
profile_text_nolan = profile_data_nolan['text'][0]
profile_doc_nolan = nlp(profile_text_nolan)

# First similarity scores for first 100 job postings
titles_and_docs['doc'][:100].map(lambda x: x.similarity(profile_doc_nolan))

0     0.799030
1     0.926522
2     0.657546
3     0.828865
4     0.828454
        ...   
95    0.794983
96    0.879577
97    0.855071
98    0.841373
99    0.809794
Name: doc, Length: 100, dtype: float64

In [195]:
# Calculate all similarity scores
nolan_scores = jobs[['job_title']].copy()
nolan_scores['sim_score'] = titles_and_docs['doc'].map(lambda x: x.similarity(profile_doc_nolan))

In [196]:
# Nolan's results
nolan_scores.groupby('job_title').agg(['count', 'mean'])['sim_score'].sort_values('mean', ascending=False)

,count,mean
job_title,,
Administrator,899,0.854444
Architect,831,0.853405
Data Position,283,0.850451
Developer,5274,0.849798
Programmer,321,0.848576
Analyst,1900,0.845167
Consulting,592,0.844738
Engineer,4118,0.844079
Designer,221,0.838389


In [202]:
# Comparison
zach_scores.groupby('job_title').agg(['mean'])['sim_score'].sort_values('mean', ascending=False).head(3)

,mean
job_title,
Data Position,0.926926
Analyst,0.925358
Engineer,0.924823


In [204]:
nolan_scores.groupby('job_title').agg(['mean'])['sim_score'].sort_values('mean', ascending=False).head(3)

,mean
job_title,
Administrator,0.854444
Architect,0.853405
Data Position,0.850451


---
**Approach from medium article**  
https://medium.com/@armandj.olivares/building-nlp-content-based-recommender-systems-b104a709c042

In [30]:
list_docs = []
for i in range(len(jobs)):
    if jobs['job_description'][i]:
        doc = nlp(jobs['job_description'][i])
        list_docs.append(doc)

In [31]:
def calculate_sim_with_spacy(nlp, df, user_text, n=6):
    list_sim = []
    doc1 = nlp("user_text")
    for i in df.index:
        try:
            doc2 = list_docs[i]
            score = doc1.similarity(doc2)
            list_sim.append((doc1, doc2, score))
        except:
            continue
    return list_sim

---

In [36]:
list_docs[0]

u'If you are an EDI Analyst with experience, please read on!We are a strong, long standing company looking for an EDI Analyst for our team. You must have 3+ years of EDI experience in a TrustedLink for i environment. Your role will work with our finance department identifying trading partners, work closely with external customers and be the EDI liaison across the company. You will also monitor AS2 and VAN communications, correct errors and incoming data.What You Need for this PositionRequirements:3+ years of EDI experience3+ years of TrustedLink for i Experience, iSeries/AS400Experience with VAN and AS2 communicationsWhat's In It for YouWe offer a strong compensation package and benefits!Local Candidates ONLY please.So, if you are an EDI Analyst with experience, please apply today!Applicants must be authorized to work in the U.S.Please apply directly to by clicking 'Click Here to Apply' with your Word resume!Looking forward to receiving your resume and going over the position in more d

In [40]:
sim = calculate_sim_with_spacy(nlp, jobs['job_description'], 'java developer')

<ipython-input-31-4bb41cda221f>:7: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  score = doc1.similarity(doc2)


In [50]:
sims = [(s[1],s[2]) for s in sim]

In [58]:
df = pd.DataFrame(sims).sort_values(by=1).reset_index(drop=True)

In [59]:
df.head(5)

,0,1
0,"(u'Please, share, resume, to, , or, Call, me,...",0.030524
1,"(u'Job, Summary, :, The, Manager, of, Informat...",0.031012
2,"(u'E*Pro, Consulting, service, offerings, incl...",0.062507
3,"(u'Role, :, Team, center, PLM, developerLocati...",0.072988
4,"(u'US, citizens, and, those, authorized, to, w...",0.104845


In [79]:
df.tail(10)

,0,1
16255,"(u'Software, Architect, 6, MonthsDearborn, ,, ...",0.545974
16256,"(u'PHP, Developer, ')",0.549071
16257,"(u'Liferay, Website, hosted, on, version, 6.2L...",0.550896
16258,"(u'5, years, experience, working, on, Middlewa...",0.573112
16259,"(u'""MusthavepriorDataconversionexperienceproje...",0.577609
16260,"(u'Network, Storage, Engineer6, monthsAustin, ...",0.579050
16261,"(u'SR, Android, Developer, ')",0.601328
16262,"(u'Java, and, Liferay, ')",0.622916
16263,"(u'systems, engineer, RF, communica, Design, t...",0.690385
16264,"(u'Java, planner, ')",0.750848
